In [1]:
import pandas as pd

In [2]:
events = pd.read_csv('events_up_to_01062018.csv', low_memory=False)
training_set = pd.read_csv('labels_training_set.csv')

In [3]:
training_set.describe()

,label
count,19414.000000
mean,0.050479
std,0.218937
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [4]:
events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2341681 entries, 0 to 2341680
Data columns (total 23 columns):
timestamp                   object
event                       object
person                      object
url                         object
sku                         float64
model                       object
condition                   object
storage                     object
color                       object
skus                        object
search_term                 object
staticpage                  object
campaign_source             object
search_engine               object
channel                     object
new_vs_returning            object
city                        object
region                      object
country                     object
device_type                 object
screen_resolution           object
operating_system_version    object
browser_version             object
dtypes: float64(1), object(22)
memory usage: 410.9+ MB


In [5]:
events['timestamp'] = pd.to_datetime(events['timestamp'])
events['brand'] = events['model'].str.split().str.get(0)
events['SO'] = events['operating_system_version'].str.split().str.get(0)
events['week'] = events.timestamp.dt.week
events['month'] = events.timestamp.dt.month
events['hour'] = events.timestamp.dt.hour
events['month_day'] = events.timestamp.dt.day
events.head(5)

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,device_type,screen_resolution,operating_system_version,browser_version,brand,SO,week,month,hour,month_day
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,Samsung,NaN,20,5,0,18
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,...,NaN,NaN,NaN,NaN,iPhone,NaN,20,5,0,18
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iPhone 6S,Muito Bom,64GB,Prateado,NaN,...,NaN,NaN,NaN,NaN,iPhone,NaN,20,5,0,18
3,2018-05-18 00:11:14,viewed product,2d681dd8,NaN,11890.0,iPhone 7,Bom,128GB,Vermelho,NaN,...,NaN,NaN,NaN,NaN,iPhone,NaN,20,5,0,18
4,2018-05-18 00:11:09,viewed product,cccea85e,NaN,7517.0,LG G4 H818P,Excelente,32GB,Branco,NaN,...,NaN,NaN,NaN,NaN,LG,NaN,20,5,0,18


In [6]:
events['event'].value_counts()

viewed product       1248124
brand listing         216312
visited site          204069
ad campaign hit       191388
generic listing       160176
searched products     130616
search engine hit     106406
checkout               65315
staticpage             11201
conversion              7091
lead                     983
Name: event, dtype: int64

In [7]:
buyers = training_set.loc[training_set['label'] == 1, ['person']]
not_buyers = training_set.loc[training_set['label'] == 0, ['person']]
print(buyers.shape)
print(not_buyers.shape)

(980, 1)
(18434, 1)


In [8]:
#Compradores
events.loc[events['person'].isin(buyers.person)]['country'].value_counts()

Brazil           8755
Unknown           142
United States      28
Netherlands         2
Name: country, dtype: int64

In [9]:
#Los no compradores
events.loc[events['person'].isin(not_buyers.person), :]['brand'].value_counts()#.groupby('event')['timestamp'].count()

iPhone      276381
Samsung     229617
Motorola     74792
LG           11351
Sony          7249
Lenovo        5059
iPad          2566
Asus          1856
Quantum        836
Name: brand, dtype: int64

## Primeros eventos de personas en el set de entrenamiento

In [10]:
firstEventsOfBuyers = events.loc[events['person'].isin(buyers.person), :].groupby('person', as_index=False).first()

firstEventsOfBuyers['new_vs_returning'].value_counts()

New          582
Returning    393
Name: new_vs_returning, dtype: int64

In [11]:
firstEventsOfNonBuyers = events.loc[events['person'].isin(not_buyers.person), :].groupby('person', as_index=False).first()

firstEventsOfNonBuyers['new_vs_returning'].value_counts()

New          13877
Returning     4274
Name: new_vs_returning, dtype: int64

In [12]:
firstEventsOfBuyers['event'].value_counts()

viewed product       534
ad campaign hit      184
search engine hit    128
searched products     62
checkout              31
conversion            28
staticpage            11
generic listing        2
Name: event, dtype: int64

In [13]:
columns_to_drop = ['timestamp', 'sku', 'storage', 'color', 'skus', 'search_term', 'staticpage', 'search_engine', 'channel', 'city', 'region', 'country', 'screen_resolution', 'operating_system_version', 'browser_version']
firstEventsOfBuyers.drop(columns_to_drop, inplace=True, axis=1)
firstEventsOfNonBuyers.drop(columns_to_drop, inplace=True, axis=1)

## Mergeando los datos de los primeros eventos de los usuarios del set de entrenamiento

In [14]:
#events['firstViewedProductModel'] = events['person'].map(lambda x: mapFirstEvent(x, 'model'))#firstEventsOfBuyers.loc[firstEventsOfBuyers['event'] == 'viewed product' & firstEventsOfBuyers.url.str.startswith('/comprar/'), :].url.str.replace('/comprar/', '')
events = pd.merge(events, pd.concat([firstEventsOfBuyers, firstEventsOfNonBuyers]), how='left', on='person', suffixes=['', '_first_event'])
#events = pd.merge(events, firstEventsOfNonBuyers, how='left', on='person', suffixes=['', '_first_event'])
events.head(5)

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,condition_first_event,campaign_source_first_event,new_vs_returning_first_event,device_type_first_event,brand_first_event,SO_first_event,week_first_event,month_first_event,hour_first_event,month_day_first_event
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,...,Muito Bom,criteo,New,Smartphone,iPhone,Android,20.0,5.0,0.0,18.0
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iPhone 6S,Muito Bom,64GB,Prateado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-05-18 00:11:14,viewed product,2d681dd8,NaN,11890.0,iPhone 7,Bom,128GB,Vermelho,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-05-18 00:11:09,viewed product,cccea85e,NaN,7517.0,LG G4 H818P,Excelente,32GB,Branco,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Agregando columna de conversions/viewed products y checkouts/viewed products y conversions/checkouts mensuales

In [15]:
def generate_person_month(person, month):
    return person.astype(str) + "_" + month.astype(str)

In [16]:
def get_value_for_month(values, person, month_no, column):
    if (len(values.loc[(values['month'] == month_no) & (values['person'] == person)]) > 0):
        return values.loc[(values['month'] == month_no) & (values['person'] == person)][column].values[0]
    else:
        return 0

In [17]:
#est = events.loc[events['event'] == 'checkout', :].groupby(['person', 'month'])['event'].agg({'total_checkouts' : 'count'}).reset_index()
#test.loc[(test['month'] == 5) & (test['person'] == '000ba417')]['total_checkouts'].values[0]
#tesa
#test['total_checkouts_january'] = get_value_for_month(test, test['person'], 1, 'total_checkouts')
#test['total_checkouts_february'] = get_value_for_month(test, test['person'], 2, 'total_checkouts')
#test['total_checkouts_march'] = get_value_for_month(test, test['person'], 3, 'total_checkouts')
#test['total_checkouts_april'] = get_value_for_month(test, test['person'], 4, 'total_checkouts')
#test['total_checkouts_may'] = get_value_for_month(test, test['person'], 5, 'total_checkouts')

In [18]:
numberOfCheckoutsPerPersonPerMonth = events.loc[events['event'] == 'checkout', :].groupby(['person', 'month'])['event'].agg({'total_checkouts' : 'count'}).reset_index()
numberOfConversionsPerPersonPerMonth = events.loc[events['event'] == 'conversions', :].groupby(['person', 'month'])['event'].agg({'total_conversions' : 'count'}).reset_index()
numberOfProductsViewedPerMonth = events.loc[events['event'] == 'viewed product', :].groupby(['person', 'month'])['event'].agg({'total_products_viewed' : 'count'}).reset_index()

/home/maximiliano/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """Entry point for launching an IPython kernel.
/home/maximiliano/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  
/home/maximiliano/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  This is separate from the ipykernel package so we can avoid doing imports until


In [19]:
#Generando columna necesaria para join
numberOfCheckoutsPerPersonPerMonth['person_month'] = generate_person_month(numberOfCheckoutsPerPersonPerMonth['person'], numberOfCheckoutsPerPersonPerMonth['month'])
numberOfConversionsPerPersonPerMonth['person_month'] = generate_person_month(numberOfConversionsPerPersonPerMonth['person'], numberOfConversionsPerPersonPerMonth['month'])
numberOfProductsViewedPerMonth['person_month'] = generate_person_month(numberOfProductsViewedPerMonth['person'], numberOfProductsViewedPerMonth['month'])
events['person_month'] = generate_person_month(events['person'], events['month'])

numberOfCheckoutsPerPersonPerMonth['total_checkouts_january'] = get_value_for_month(numberOfCheckoutsPerPersonPerMonth, numberOfCheckoutsPerPersonPerMonth['person'], 1, 'total_checkouts')
numberOfCheckoutsPerPersonPerMonth['total_checkouts_february'] = get_value_for_month(numberOfCheckoutsPerPersonPerMonth, numberOfCheckoutsPerPersonPerMonth['person'], 2, 'total_checkouts')
numberOfCheckoutsPerPersonPerMonth['total_checkouts_march'] = get_value_for_month(numberOfCheckoutsPerPersonPerMonth, numberOfCheckoutsPerPersonPerMonth['person'], 3, 'total_checkouts')
numberOfCheckoutsPerPersonPerMonth['total_checkouts_april'] = get_value_for_month(numberOfCheckoutsPerPersonPerMonth, numberOfCheckoutsPerPersonPerMonth['person'], 4, 'total_checkouts')
numberOfCheckoutsPerPersonPerMonth['total_checkouts_may'] = get_value_for_month(numberOfCheckoutsPerPersonPerMonth, numberOfCheckoutsPerPersonPerMonth['person'], 5, 'total_checkouts')

numberOfConversionsPerPersonPerMonth['total_conversions_january'] = get_value_for_month(numberOfConversionsPerPersonPerMonth, numberOfConversionsPerPersonPerMonth['person'], 1, 'total_conversions')
numberOfConversionsPerPersonPerMonth['total_conversions_february'] = get_value_for_month(numberOfConversionsPerPersonPerMonth, numberOfConversionsPerPersonPerMonth['person'], 2, 'total_conversions')
numberOfConversionsPerPersonPerMonth['total_conversions_march'] = get_value_for_month(numberOfConversionsPerPersonPerMonth, numberOfConversionsPerPersonPerMonth['person'], 3, 'total_conversions')
numberOfConversionsPerPersonPerMonth['total_conversions_april'] = get_value_for_month(numberOfConversionsPerPersonPerMonth, numberOfConversionsPerPersonPerMonth['person'], 4, 'total_conversions')
numberOfConversionsPerPersonPerMonth['total_conversions_may'] = get_value_for_month(numberOfConversionsPerPersonPerMonth, numberOfConversionsPerPersonPerMonth['person'], 5, 'total_conversions')

numberOfProductsViewedPerMonth['total_products_viewed_january'] = get_value_for_month(numberOfProductsViewedPerMonth, numberOfProductsViewedPerMonth['person'], 1, 'total_products_viewed')
numberOfProductsViewedPerMonth['total_products_viewed_february'] = get_value_for_month(numberOfProductsViewedPerMonth, numberOfProductsViewedPerMonth['person'], 2, 'total_products_viewed')
numberOfProductsViewedPerMonth['total_products_viewed_march'] = get_value_for_month(numberOfProductsViewedPerMonth, numberOfProductsViewedPerMonth['person'], 3, 'total_products_viewed')
numberOfProductsViewedPerMonth['total_products_viewed_april'] = get_value_for_month(numberOfProductsViewedPerMonth, numberOfProductsViewedPerMonth['person'], 4, 'total_products_viewed')
numberOfProductsViewedPerMonth['total_products_viewed_may'] = get_value_for_month(numberOfProductsViewedPerMonth, numberOfProductsViewedPerMonth['person'], 5, 'total_products_viewed')


numberOfCheckoutsPerPersonPerMonth.drop(['person', 'month'], axis=1, inplace=True)
numberOfConversionsPerPersonPerMonth.drop(['person', 'month'], axis=1, inplace=True)
numberOfProductsViewedPerMonth.drop(['person', 'month'], axis=1, inplace=True)

monthTotals = pd.concat([numberOfCheckoutsPerPersonPerMonth, numberOfConversionsPerPersonPerMonth, numberOfProductsViewedPerMonth])
monthTotals.head()

/home/maximiliano/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:30: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



,person_month,total_checkouts,total_checkouts_april,total_checkouts_february,total_checkouts_january,total_checkouts_march,total_checkouts_may,total_conversions,total_conversions_april,total_conversions_february,total_conversions_january,total_conversions_march,total_conversions_may,total_products_viewed,total_products_viewed_april,total_products_viewed_february,total_products_viewed_january,total_products_viewed_march,total_products_viewed_may
0,0008ed71_5,3.0,2.0,9.0,10.0,1.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00091926_5,2.0,2.0,9.0,10.0,1.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,000ba417_5,6.0,2.0,9.0,10.0,1.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,000c79fe_5,1.0,2.0,9.0,10.0,1.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,000e4d9e_5,1.0,2.0,9.0,10.0,1.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
#events = pd.merge(events, numberOfCheckoutsPerPersonPerMonth, how='left', on='person_month')
#events = pd.merge(events, numberOfConversionsPerPersonPerMonth, how='left', on='person_month')
#events = pd.merge(events, numberOfProductsViewedPerMonth, how='left', on='person_month')
#events.head(5)

In [21]:
#events.drop(['person_month'], axis=1, inplace=True)
#events.head(5)

In [22]:
events.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2341681 entries, 0 to 2341680
Data columns (total 43 columns):
timestamp                       datetime64[ns]
event                           object
person                          object
url                             object
sku                             float64
model                           object
condition                       object
storage                         object
color                           object
skus                            object
search_term                     object
staticpage                      object
campaign_source                 object
search_engine                   object
channel                         object
new_vs_returning                object
city                            object
region                          object
country                         object
device_type                     object
screen_resolution               object
operating_system_version        object
browser_version     

## Trabajando con las columnas y pasandolas a labels

In [23]:
eventsRegions = events['region'].value_counts()
topRegions = eventsRegions.head(5)
secondTopRegions = eventsRegions.iloc[5:18]
restOfTheRegions = eventsRegions.drop(eventsRegions.index[0:18])
topRegionsDF = pd.DataFrame({'region':topRegions.index, 'region_no':3})
secondTopRegionsDF = pd.DataFrame({'region':secondTopRegions.index, 'region_no':2})
restOfTheRegionsDF = pd.DataFrame({'region':restOfTheRegions.index, 'region_no':1})
regionsMerge = pd.concat([topRegionsDF, secondTopRegionsDF, restOfTheRegionsDF])

In [24]:
events = pd.merge(events, regionsMerge, how='left', on='region')
events.drop('region', axis=1, inplace=True)
events.head(5)

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,new_vs_returning_first_event,device_type_first_event,brand_first_event,SO_first_event,week_first_event,month_first_event,hour_first_event,month_day_first_event,person_month,region_no
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4886f805_5,NaN
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,...,New,Smartphone,iPhone,Android,20.0,5.0,0.0,18.0,ad93850f_5,NaN
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iPhone 6S,Muito Bom,64GB,Prateado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0297fc1e_5,NaN
3,2018-05-18 00:11:14,viewed product,2d681dd8,NaN,11890.0,iPhone 7,Bom,128GB,Vermelho,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2d681dd8_5,NaN
4,2018-05-18 00:11:09,viewed product,cccea85e,NaN,7517.0,LG G4 H818P,Excelente,32GB,Branco,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cccea85e_5,NaN


In [25]:
conditionsDictionary = {
            "Novo": 1,
            "Bom - Sem Touch ID": 2,
            "Muito Bom": 3,
            "Excelente": 4,
            "Bom": 5}
events['condition_no'] = events['condition'].map(lambda x: conditionsDictionary.get(x))
events.head(5)

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,device_type_first_event,brand_first_event,SO_first_event,week_first_event,month_first_event,hour_first_event,month_day_first_event,person_month,region_no,condition_no
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4886f805_5,NaN,4.0
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,...,Smartphone,iPhone,Android,20.0,5.0,0.0,18.0,ad93850f_5,NaN,3.0
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iPhone 6S,Muito Bom,64GB,Prateado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0297fc1e_5,NaN,3.0
3,2018-05-18 00:11:14,viewed product,2d681dd8,NaN,11890.0,iPhone 7,Bom,128GB,Vermelho,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2d681dd8_5,NaN,5.0
4,2018-05-18 00:11:09,viewed product,cccea85e,NaN,7517.0,LG G4 H818P,Excelente,32GB,Branco,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cccea85e_5,NaN,4.0


In [26]:
newVsReturningDictionary = {
            "Returning": 1,
            "New": 2}
events['new_vs_returning'] = events['new_vs_returning'].map(lambda x: newVsReturningDictionary.get(x))
events['new_vs_returning_first_event'] = events['new_vs_returning_first_event'].map(lambda x: newVsReturningDictionary.get(x))
events.head(5)

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,device_type_first_event,brand_first_event,SO_first_event,week_first_event,month_first_event,hour_first_event,month_day_first_event,person_month,region_no,condition_no
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4886f805_5,NaN,4.0
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,...,Smartphone,iPhone,Android,20.0,5.0,0.0,18.0,ad93850f_5,NaN,3.0
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iPhone 6S,Muito Bom,64GB,Prateado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0297fc1e_5,NaN,3.0
3,2018-05-18 00:11:14,viewed product,2d681dd8,NaN,11890.0,iPhone 7,Bom,128GB,Vermelho,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2d681dd8_5,NaN,5.0
4,2018-05-18 00:11:09,viewed product,cccea85e,NaN,7517.0,LG G4 H818P,Excelente,32GB,Branco,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cccea85e_5,NaN,4.0


In [27]:
eventCountries = events['country'].value_counts()
topCountries = eventCountries.head(1)
secondTopCountries = eventCountries.iloc[1:2]
thirdTopCountries = eventCountries.iloc[2:3]
restOfTheCountries = eventCountries.drop(eventCountries.index[0:4])

topCountriesDF = pd.DataFrame({'country':topRegions.index, 'country_no':4})
secondTopCountriesDF = pd.DataFrame({'country':secondTopCountries.index, 'country_no':3})
thirdTopCountriesDF = pd.DataFrame({'country':thirdTopCountries.index, 'country_no':2})
restOfTheCountriesDF = pd.DataFrame({'country':restOfTheCountries.index, 'country_no':1})
countriesMerge = pd.concat([topCountriesDF, secondTopCountriesDF, thirdTopCountriesDF, restOfTheCountriesDF])

In [28]:
events = pd.merge(events, countriesMerge, how='left', on='country')
events.drop('country', axis=1, inplace=True)
events.head(5)

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,brand_first_event,SO_first_event,week_first_event,month_first_event,hour_first_event,month_day_first_event,person_month,region_no,condition_no,country_no
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4886f805_5,NaN,4.0,NaN
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,...,iPhone,Android,20.0,5.0,0.0,18.0,ad93850f_5,NaN,3.0,NaN
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iPhone 6S,Muito Bom,64GB,Prateado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0297fc1e_5,NaN,3.0,NaN
3,2018-05-18 00:11:14,viewed product,2d681dd8,NaN,11890.0,iPhone 7,Bom,128GB,Vermelho,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2d681dd8_5,NaN,5.0,NaN
4,2018-05-18 00:11:09,viewed product,cccea85e,NaN,7517.0,LG G4 H818P,Excelente,32GB,Branco,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,cccea85e_5,NaN,4.0,NaN


In [29]:
eventsDictionary = {
            "lead": 1,
            "visited site": 2,
            "brand listing": 3,
            "generic listing": 4,
            "staticpage": 5,
            "conversion": 6,
            "checkout": 7,
            "searched products": 8,
            "search engine hit": 9,
            "ad campaign hit": 10,
            "viewed product": 11,
            }
events['event_first_event'] = events['event_first_event'].map(lambda x: eventsDictionary.get(x))
events.head(5)

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,brand_first_event,SO_first_event,week_first_event,month_first_event,hour_first_event,month_day_first_event,person_month,region_no,condition_no,country_no
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4886f805_5,NaN,4.0,NaN
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,...,iPhone,Android,20.0,5.0,0.0,18.0,ad93850f_5,NaN,3.0,NaN
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iPhone 6S,Muito Bom,64GB,Prateado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0297fc1e_5,NaN,3.0,NaN
3,2018-05-18 00:11:14,viewed product,2d681dd8,NaN,11890.0,iPhone 7,Bom,128GB,Vermelho,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2d681dd8_5,NaN,5.0,NaN
4,2018-05-18 00:11:09,viewed product,cccea85e,NaN,7517.0,LG G4 H818P,Excelente,32GB,Branco,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,cccea85e_5,NaN,4.0,NaN


In [30]:
events.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2346954 entries, 0 to 2346953
Data columns (total 44 columns):
timestamp                       datetime64[ns]
event                           object
person                          object
url                             object
sku                             float64
model                           object
condition                       object
storage                         object
color                           object
skus                            object
search_term                     object
staticpage                      object
campaign_source                 object
search_engine                   object
channel                         object
new_vs_returning                float64
city                            object
device_type                     object
screen_resolution               object
operating_system_version        object
browser_version                 object
brand                           object
SO                 

In [31]:
brandsDictionary = {
            "Xiaomi": 1,
            "Outros": 2,
            "Asus": 3,
            "iPad": 4,
            "Lenovo": 5,
            "Sony": 6,
            "LG": 7,
            "Motorola": 8,
            "Samsung": 9,
            "iPhone": 10}
#events['brand'] = events['brand'].map(lambda x: brandsDictionary.get(x))
one_hot = pd.get_dummies(events['brand'])
events = events.drop('brand', axis=1)
events = events.join(one_hot)

In [32]:
events.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2346954 entries, 0 to 2346953
Data columns (total 54 columns):
timestamp                       datetime64[ns]
event                           object
person                          object
url                             object
sku                             float64
model                           object
condition                       object
storage                         object
color                           object
skus                            object
search_term                     object
staticpage                      object
campaign_source                 object
search_engine                   object
channel                         object
new_vs_returning                float64
city                            object
device_type                     object
screen_resolution               object
operating_system_version        object
browser_version                 object
SO                              object
week               

## Agregando el producto mas visitado en los eventos de cada persona

In [33]:
def most_viewed_model(group):
    top_2_most_viewed = group.value_counts().head(2)
    if len(top_2_most_viewed.index) > 0:
        if top_2_most_viewed.index[0] != 'idk' or len(top_2_most_viewed) == 1:
            return top_2_most_viewed.index[0]
        else:
            return top_2_most_viewed.index[1]
    else:
        return None

In [34]:
e = events.groupby('person').model.apply(most_viewed_model).reset_index()
e = e.rename(columns=dict(level_2='model', model='most_viewed_model'))
events = events.merge(e, how='inner', on='person')

In [35]:
events['most_viewed_brand'] = events['model'].str.split().str.get(0)
events['most_viewed_brand'] = events['most_viewed_brand'].map(lambda x: brandsDictionary.get(x))

In [36]:
soDictionary = {
            "FreeBSD": 1,
            "Symbian": 2,
            "Fedora": 3,
            "Tizen": 4,
            "Blackberry": 5,
            "Other": 6,
            "Chrome": 7,
            "Ubuntu": 8,
            "Linux": 9,
            "Mac": 10,
            "iOS": 11,
            "Windows": 12,
            "Android": 13}
#events['SO'] = events['SO'].map(lambda x: soDictionary.get(x))
one_hot = pd.get_dummies(events['SO'])
events = events.drop('SO', axis=1)
events = events.join(one_hot)

In [37]:
deviceTypesDictionary = {
            "Unknown": 1,
            "Tablet": 2,
            "Computer": 3,
            "Smarthone": 4}
#events['device_type'] = events['device_type'].map(lambda x: deviceTypesDictionary.get(x))
one_hot = pd.get_dummies(events['device_type'])
events = events.drop('device_type', axis=1)
events = events.join(one_hot)

## Borrando columnas

In [38]:
events.drop(['url', 'sku', 'operating_system_version', 'browser_version', 'condition', 'skus', 'storage', 'timestamp', 'screen_resolution', 'color', 'search_engine', 'staticpage', 'search_term', 'campaign_source', 'channel', 'city'], axis=1, inplace=True)
events.head(10)

,event,person,model,new_vs_returning,week,month,hour,month_day,event_first_event,url_first_event,...,Other,Symbian,Tizen,Ubuntu,Windows,iOS,Computer,Smartphone,Tablet,Unknown
0,viewed product,4886f805,Samsung Galaxy J7 Prime,NaN,20,5,0,18,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
1,viewed product,4886f805,Samsung Galaxy J7 Prime,NaN,20,5,0,18,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2,search engine hit,4886f805,NaN,NaN,20,5,0,18,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
3,checkout,4886f805,Samsung Galaxy J7 Prime,NaN,20,5,0,18,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
4,viewed product,4886f805,Samsung Galaxy J7 Prime,NaN,20,5,0,18,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
5,searched products,4886f805,NaN,NaN,20,5,0,18,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
6,viewed product,4886f805,Samsung Galaxy J7 Prime,NaN,20,5,0,18,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
7,generic listing,4886f805,NaN,NaN,20,5,0,18,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
8,visited site,4886f805,NaN,2.0,20,5,0,18,NaN,NaN,...,0,0,0,0,0,0,0,1,0,0
9,viewed product,ad93850f,iPhone 5s,NaN,20,5,0,18,11.0,/comprar/iphone/iphone-5s,...,0,0,0,0,0,0,0,0,0,0


In [39]:
one_hot = pd.get_dummies(events['event'])
events = events.drop('event', axis=1)
events = events.join(one_hot)

In [40]:
events.fillna(0, inplace=True)
events.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2346954 entries, 0 to 2346953
Data columns (total 65 columns):
person                          object
model                           object
new_vs_returning                float64
week                            int64
month                           int64
hour                            int64
month_day                       int64
event_first_event               float64
url_first_event                 object
model_first_event               object
condition_first_event           object
campaign_source_first_event     object
new_vs_returning_first_event    float64
device_type_first_event         object
brand_first_event               object
SO_first_event                  object
week_first_event                float64
month_first_event               float64
hour_first_event                float64
month_day_first_event           float64
person_month                    object
region_no                       float64
condition_no             

In [44]:
events.info(5)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4251337 entries, 0 to 4251336
Data columns (total 85 columns):
person                                 object
model                                  object
new_vs_returning                       float64
week                                   int64
month                                  int64
hour                                   int64
month_day                              int64
event_first_event                      float64
url_first_event                        object
model_first_event                      object
condition_first_event                  object
campaign_source_first_event            object
new_vs_returning_first_event           float64
device_type_first_event                object
brand_first_event                      object
SO_first_event                         object
week_first_event                       float64
month_first_event                      float64
hour_first_event                       float64
month_day_f

## Uniendo las cantidades de conversiones, checkouts y productos vistos y agregando columnas de promedios

In [42]:
events = pd.merge(events, monthTotals, how='left', on='person_month')

events['conversions_vs_checkouts_mean'] = events['total_conversions']/events['total_checkouts']
events['checkouts_vs_products_viewed_mean'] = events['total_checkouts']/events['total_products_viewed']
events['conversions_vs_products_viewed_mean'] = events['total_conversions']/events['total_products_viewed']

events.drop('person_month', inplace=True, axis=1)
events.head()

,person,model,new_vs_returning,week,month,hour,month_day,event_first_event,url_first_event,model_first_event,...,total_conversions_may,total_products_viewed,total_products_viewed_april,total_products_viewed_february,total_products_viewed_january,total_products_viewed_march,total_products_viewed_may,conversions_vs_checkouts_mean,checkouts_vs_products_viewed_mean,conversions_vs_products_viewed_mean
0,4886f805,Samsung Galaxy J7 Prime,0.0,20,5,0,18,0.0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4886f805,Samsung Galaxy J7 Prime,0.0,20,5,0,18,0.0,0,0,...,NaN,4.0,38.0,22.0,6.0,3.0,372.0,NaN,NaN,NaN
2,4886f805,Samsung Galaxy J7 Prime,0.0,20,5,0,18,0.0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4886f805,Samsung Galaxy J7 Prime,0.0,20,5,0,18,0.0,0,0,...,NaN,4.0,38.0,22.0,6.0,3.0,372.0,NaN,NaN,NaN
4,4886f805,0,0.0,20,5,0,18,0.0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Generando csv

In [43]:
#events.to_csv('setEntrenamientoM.csv')